   # Sandbox: Ignore this
   This notebook contains little experiments on doing things with python, jupyter, ipython et cetera
   It's not a central part of the project.
    
   Mostly ad hoc code snippets.  And not documented well

In [7]:
import os
import sys
import time
import datetime
import touch
import inspect
import datetime
# don't really need these below, just for testing
import numpy
from PIL import Image 
import plotnine as pn
import pandas as pd
print(sys.version_info)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


In [3]:
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
        continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.8.5 of Python
No __version__ for touch as touch
version numpy of numpy as 1.19.3 
version Image of Image as 8.0.1 
version pn of plotnine as 0.7.1 
version pd of pandas as 1.1.3 


In [4]:
# snippet to see the source code for a fuction
import inspect
import nbformat
fname = "nbformat.convert"
lines = inspect.getsource(eval(fname))
#print(lines)

In [5]:
# to get the names of notebook files in a directory
def get_project_notebooks():
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    proj_nbs = []
    nb_fnames = [f for f in os.listdir() if f.endswith(".ipynb")]
    for fn in nb_fnames:
        with open(fn) as fp:
            notebook = read(fp, as_version=NO_CONVERT)
        cells = notebook['cells']
        if not cells:
            continue
        src =cells[0]["source"]
        if not src:
            continue
        lines = src.split("\n")
        if lines[0].startswith("# "):
            proj_nbs.append(fn)
    return(proj_nbs)


# print the source code for a cell
def print_source(cell):
    print("type cell= {0}".format(type(cell)))
    src = cell["source"]
    lines = src.split("\n")
    print("{0} lines".format(len(lines)))
    for i,line in enumerate(lines):
        print("({0}){1}".format(i, line))

# run notebook code
def run_nb_code(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    for i, cell in enumerate(code_cells):
        src = cell['source']
        #print("{0}<{1}>".format(li, line))                
        try:
            exec(src, globals(), locals())
        except Exception as e:
            print("Error on cell {0}\n".format(i))
            print(src)
            print_source(cell)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            import traceback
            traceback.print_tb(exc_tb)
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno)
            return((exc_type, exc_obj, exc_tb, fname))
    res += "end: " + str(datetime.datetime.now())
    return res
# run notebook code
def get_first_cell_line(nbfile):
    from nbformat import read, NO_CONVERT
    import os
    import datetime
    res = "Start time: " + str(datetime.datetime.now())
    with open(nbfile) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    if not cells:
        return ""
    lines = cells[0]["source"].split("\n")
    if not lines:
        return ""
    return lines[0]
def ordered_notebooks():
    nbs = get_project_notebooks()
    nbord = [None]*len(nbs)
    end_index = len(nbs)-1
    for nb in nbs:
        line0 = get_first_cell_line(nb)
        parts = line0.split(' ')

        index = -1
        if len(parts) >= 3 and parts[0]=='#' and parts[1]=='Part':
            try:
                index = int(parts[2]) - 1
            except Exception as e:
                pass
        if  index == -1:
            index = end_index
            end_index -= 1
        nbord[index] = nb
    return (nbord)
print(get_project_notebooks())
print(ordered_notebooks())

['combining_claims.ipynb', 'DataSources.ipynb', 'fastpages-howto.ipynb', 'fred.ipynb', 'google.ipynb', 'Intro_Overview.ipynb', 'setup.ipynb', 'Setup_old.ipynb', 'umich_cs.ipynb', 'weekly_claims.ipynb']
['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb', 'Setup_old.ipynb', 'setup.ipynb', 'fastpages-howto.ipynb']


In [3]:
class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""
    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

In [27]:
from IPython.utils.capture import CapturedIO
from IPython.utils.capture import capture_output
nbs = ordered_notebooks()
print(nbs)
ofname = 'foo_out.txt'
with open(ofname, mode='w') as fp:
    fp.write("Datetime= {0}".format(str(datetime.datetime.now())))
outstr = ""
x = None
for nb in nbs:
    print("\nRunnning {0}".format(nb))
    with capture_output() as co:
        %run  $nb
        ofname = os.path.splitext(nb)[0] + ".out"
        if nb == 'fred.ipynb':
            x = co
            break
        with open(ofname, mode="w") as fp:
            fp.write(str(datetime.datetime.now()))
            fp.write(" stdout\n{0}\ns stderr{1}".format(co.stdout, co.stderr))
            for o in co.outputs:
                fp.write(str(o))

['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb', 'Setup_old.ipynb', 'setup.ipynb', 'fastpages-howto.ipynb']

Runnning Intro_Overview.ipynb

Runnning DataSources.ipynb

Runnning fred.ipynb


In [36]:
x.outputs
repr(x.stdout)

"'version 3.8.5 of Python\\nversion 2.2.1 of re\\nversion 1.19.3 of numpy\\nversion 8.0.1 of Image\\nversion 0.7.1 of pn\\nversion 1.1.3 of pd\\nversion 1.19.3 of np\\nversion 3.3.3 of mpl\\nversion 0.4.2 of fredapi\\nversion 1.2.0 of xlrd\\nversion 3.141.0 of selenium\\nversion 3.14.1 of webdriver\\nversion 2020.1 of pytz\\n0.4.2\\nUSPRIV, rows= 166\\nNPPTTL, rows= 166\\nICSA, rows= 725\\nCCSA, rows= 724\\nJTS1000JOL, rows= 165\\nJTS1000HIL, rows= 165\\nJTS1000TSL, rows= 165\\nCSCICP03USM665S, rows= 166\\nBSCICP03USM665S, rows= 166\\nUNRATE, rows= 166\\nUNEMPLOY, rows= 166\\nCLF16OV, rows= 166\\nUEMP27OV, rows= 166\\nU6Rate, rows= 166\\nCIVPART, rows= 166\\nLNS12032194, rows= 166\\n     realtime_start        date      value series_id         description  \\\\\\n1779     2020-11-19  2020-11-07  6.372e+06      CCSA   Continued Claims    \\n1780     2020-11-25  2020-11-14  6.071e+06      CCSA   Continued Claims    \\n1055     2020-11-19  2020-11-14     742000      ICSA      Initial Claim

In [ ]:
all_nb_filenames = [f for f in os.listdir() if f.endswith(".ipynb")]
print(all_nb_filenames)
proj_nb_filenames = ['fred.ipynb', 'combiningData.ipynb', 'DataSources.ipynb', 'fastpages-howto.ipynb', 
                     'foo.ipynb', 'fred.ipynb', 'google.ipynb', 'Intro_Overview.ipynb', 'umich_cs.ipynb', 'weekly_claims.ipynb']

In [8]:
# Touch them in order
import time
nbs = get_project_notebooks()
last  = len(nbs) - 1
nblist = [""]*len(nbs)
for nb in nbs:
    line = get_first_cell_line(nb)
    line_parts = line.split(" ")
    if len(line_parts) > 3 and line_parts[1] == "Part":
        try:
            index = int(line_parts[2]) - 1
        except:
            index = last
            last -= 1
    else:
        index = last
        last -= 1
    nblist[index] = nb
    #print(f"{nb}, index={index}, {line}")
print(nblist)
def add_str_to_file(fname, str="\n"):
    with open(fname, mode='a') as fp:
        fp.write(str)
    return
if False:
    for nb in nblist:
        try:
            print("{0} {1}".format(nb, datetime.datetime.now()))
            touch.touch(nb)
            add_str_to_file(nb, str="\n")
            time.sleep(61)
        except Exception as e:
            print(e)

['Intro_Overview.ipynb', 'DataSources.ipynb', 'fred.ipynb', 'weekly_claims.ipynb', 'umich_cs.ipynb', 'google.ipynb', 'combining_claims.ipynb', 'Setup_old.ipynb', 'setup.ipynb', 'fastpages-howto.ipynb']


In [9]:
import IPython
app = IPython.Application.instance()
dir(app.kernel)
app.kernel.do_shutdown(restart=True)
#app.kernel.do_shutdown(True) 
print(datetime.datetime.now())

2020-11-30 21:13:07.587282


In [1]:
import re
head, tail = os.path.split(downloaded_fpath)
base, ext = os.path.splitext(tail)
pattern = "(.*)(\(.*\))"
match = re.match(pattern, base)
if not match:
    msg = "Error cleaning {0}\nNo match for pattern: <{1}>".format(fpath, pattern)
    raise Exception(msg)
new_base = match.group(1).strip()
new_fname = new_base + ext
return new_fname

NameError: name 'os' is not defined

In [ ]:
def print_vars(globs=True, locs=True, 
               filters={"exclude":("^_",)}):
    import re
    dicts = {"globs":globals().keys(),
            "locs":locals().keys()}
    for k,v in dicts.items():
        print("k={0}, eval(k)= {1}".format(k, eval(k)))
        if eval(k):
            y = [x for x in dicts[k]]
            if filters:
                if filters["exclude"]:
                    for filt in filters["exclude"]:
                        print("excluding: {0}".format(filt))
                        y = [x for x in y if not re.search(filt, x)]
            print("{0}= {1}".format(k,y))
        del y
print_vars()

In [ ]:
z